In [122]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time

import warnings
warnings.filterwarnings("ignore")


from sklearn.ensemble import RandomForestRegressor


table = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/starch-sanchez-v3/matrices/A6.pkl")
tableg = table.T

B = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/starch-sanchez-v3/matrices/B6.pkl")
Bdf = pd.DataFrame(B)


tableg = table.T
species_ones = [] 
for i in range(6):
    species = i
    species_ones.append(list(tableg[tableg[species]==1].index))
    
    
species_zeros = []
for i in range(6):
    species = i
    species_zeros.append(list(tableg[tableg[species]==0].index))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       


In [123]:

from sklearn.metrics import r2_score


import matplotlib.patches as mpatches

from sklearn.metrics import mean_absolute_percentage_error as mape

In [124]:
from scipy import stats

In [125]:
sp_list_provided = ["C","E","M","P","S","T"]

sstv3 = pd.read_pickle("~/compressed_sensingv1/realdatasets/starch-sanchez-v5/starch-sanchez-relative.pkl")

wellsp = list(sstv3.index)

In [126]:
#table[wellsp]

In [127]:
dataTR = pd.read_pickle("~/compressed_sensingv1/realdatasets/starch-sanchez-v5/traindataRELss.pkl")
dataTE = pd.read_pickle("~/compressed_sensingv1/realdatasets/starch-sanchez-v5/testdataRELss.pkl")

In [128]:
cv_splitsl = [3,5] 
max_depthl = [2,5,10,15,20]
ntrees = [100,200,300,400,500,1000]
max_featuresl = ["auto","sqrt","log2"]

In [129]:
rdf = pd.read_pickle("RF-REL-per.pkl")

In [130]:
rndf  = rdf[["species","maxdepth","ntrees","ypred","yobs","k-fold","features"]]

In [131]:
crossval = [] 
for m in cv_splitsl:
    samp = rndf[rndf["k-fold"]==m]
    for speciesc in range(6):
        species = speciesc
        df2 = pd.DataFrame(species_ones[speciesc])
        idxlist = list(df2[0].values)
        abunvec = pd.DataFrame(sstv3[sp_list_provided[speciesc]][list(set(wellsp).intersection(species_ones[speciesc]))])
        pp = list(abunvec.index)
        redlist = []
        for kk in pp:
            redlist.append(idxlist.index(kk))
        abunvec.index = redlist
        sp1 = samp[samp["species"]==species]
        for md in max_depthl:
            mdf = sp1[sp1["maxdepth"]==md]
            for n_est in ntrees:
                ntr = mdf[mdf["ntrees"]==n_est]
                for mfeatures in max_featuresl:
                    mfeats = ntr[ntr["features"]==mfeatures]
                    r2 = [] 
                    yallbest  = []
                    yallact  = [] 
                    mfeats.index = np.arange(0,len(mfeats),1)
                    for iterii in range(m):
                        yact = mfeats["yobs"][iterii]
                        ybest = mfeats["ypred"][iterii]
                        ybestdf1 = pd.DataFrame(ybest)
                        ybestdf1[ybestdf1[0]<1e-10]=0
                        slope, intercept,r_value, p_value, std_err = stats.linregress(yact.astype(float),ybestdf1[0].values)
                        yallbest.append(ybestdf1[0].values)
                        yallact.append(yact)
                        rval = r2_score(yact,ybestdf1[0].values)
                        gop = 1/(2-rval)
                        mape1 = mape(yact,ybestdf1[0].values)
                        r2.append([gop,rval,mape1,slope,intercept,r_value**2])
                    r2df = pd.DataFrame(r2)
                    if m == 2:
                        ybestagg = [*yallbest[0],*yallbest[1]]
                        yactagg = [*yallact[0],*yallact[1]]
                    if m == 3:
                        ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2]]
                        yactagg = [*yallact[0],*yallact[1],*yallact[2]]
                    if m == 5:
                        ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4]]
                        yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4]]
                    if m == 7:
                        ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6]]
                        yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6]]
                    slope, intercept,r_value, p_value, std_err = stats.linregress(yactagg,ybestagg)
                    rval2 = r2_score(yactagg,ybestagg)
                    gop2 = 1/(2-rval2)
                    mean1 = r2df.mean().values[0]
                    mean2 = r2df.mean().values[1]
                    mean3 = r2df.mean().values[2]
                    mean4 = r2df.mean().values[3]
                    mean5 = r2df.mean().values[4]
                    mean6 = r2df.mean().values[5]
                    mape2 = mape(yactagg,ybestagg)
                    crossval.append([m,species,md,mfeatures,n_est,gop2,rval2,mape2,slope,intercept,r_value**2,"break",mean1,mean2,mean3,mean4,mean5,mean6,yactagg,ybestagg])

In [132]:
crossdf = pd.DataFrame(crossval,columns=["k-fold","sp","max_depth","features","trees","nnseagg","r2agg","mapeagg","slagg","intagg","rlinagg","means on right","nnseavg","r2avg","mapeavg","slavg","intavg","rlinavg","testact","testpred"])

In [141]:
crossdf.to_pickle("RF-SS-analysis.pkl")

In [148]:
#cv3 = crossdf[crossdf["k-fold"]==3]
#bestbynnse = [] 
#for species in range(6):
#    spdf = cv3[cv3["sp"]==species]
#    bidx = spdf["nnseagg"].idxmax()
#    bestbynnse.append(spdf.T[bidx].T[["max_depth","features","trees","nnseagg","sp"]])

In [149]:
#perdff2 = pd.DataFrame(bestbynnse)

In [152]:
#perdff2

In [173]:
cv3 = crossdf[crossdf["k-fold"]==3]
featset = cv3[cv3["features"]=="sqrt"]
treeset = featset[featset["trees"]==100]

In [174]:
cv3 = crossdf[crossdf["k-fold"]==3]
bestbynnse = [] 
for species in range(6):
    spdf = treeset[treeset["sp"]==species]
    bidx = spdf["nnseagg"].idxmax()
    bestbynnse.append(spdf.T[bidx].T[["max_depth","features","trees","nnseagg","sp"]])

In [175]:
perdff2 = pd.DataFrame(bestbynnse)

In [178]:
perdff2.to_pickle("SS-per-RF.pkl")

In [177]:
perdff2

,max_depth,features,trees,nnseagg,sp
1,2,sqrt,100,0.472148,0
91,2,sqrt,100,0.478077,1
199,5,sqrt,100,0.670191,2
325,15,sqrt,100,0.572237,3
379,5,sqrt,100,0.455349,4
505,15,sqrt,100,0.730749,5


In [160]:
RandomForestRegressor?

Init signature:
RandomForestRegressor(
    n_estimators=100,
    *,
    criterion='squared_error',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=1.0,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
)
Docstring:     
A random forest regressor.

A random forest is a meta estimator that fits a number of classifying
decision trees on various sub-samples of the dataset and uses averaging
to improve the predictive accuracy and control over-fitting.
The sub-sample size is controlled with the `max_samples` parameter if
`bootstrap=True` (default), otherwise the whole dataset is used to build
each tree.

Read more in the :ref:`User Guide <forest>`.

Parameters
----------
n_estimators : int, default=100
    The number of trees in the forest.

    .. versioncha

In [162]:
import sklearn
sklearn.__version__

'1.1.1'